In [1]:
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException,StaleElementReferenceException, UnexpectedAlertPresentException

from bs4 import BeautifulSoup as bs
import pandas as pd
import json

options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "../chromedriver.exe"
driver = webdriver.Chrome(chromedriver_path, options=options)

region_list = []
rating_list = []
category_list = []
review_list = []


def main():
    global driver, load_wb, review_num
    
    driver.implicitly_wait(4)  # 페이지가 렌더링 될 때까지 기다림
    driver.get('https://map.kakao.com')
    
    # 검색할 목록
    regions = ['성북구','중랑구','서대문구','동대문구','서초구','양천구','구로구',
                '용산구','성동구','도봉구','중구','노원구','강서구','광진구','마포구',
                '영등포구','강동구','금천구','강북구','강남구','은평구','종로구','동작구',
                '관악구','송파구', '광주', '대전', '울산', '세종', '수원', '성남', '고양',
                '용인','부천', '안양', '남양주', '화성', '평택', '의정부', '파주',
                '김포', '군포', '광주', '이천', '양주', '구리','하남', '여주','과천',
                '가평', '춘천', '원주', '강릉', '속초','청주', '충주', '제천', '천안', '공주',
                '제천', '천안', '공주', '보령', '전주', '군산', '익산', '목포', '여수', '순천', 
                '포항', '경주', '김천', '창원', '진주', '통영', '제주', '서귀포시']
 
    # regions = ['성동구', '송파구', '영등포구']
    
    for i, region in enumerate(regions):
        # delay
        if i % 4 == 0 and i !=0 :
            sleep(5)
        print("####", i)
        search_store(region)
        
    driver.quit()
    print("finish")
    
    
def search_store(region):
    global driver
    
    region=region
    
    search_box = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색창
    search_box.send_keys(region + " 음식점")  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)
    
    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 store list 읽기
    html = driver.page_source
    soup = bs(html, 'html.parser')
    store_lst = soup.select('.placelist > .PlaceItem')  # 검색된 가게 목록
    
    # 검색된 첫 페이지 가게 목록 크롤링하기
    crawl_store(region, store_lst)
    search_box.clear()
    
    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)  # 2페이지로 자동으로 넘어감
        sleep(1)
    
        # 2 ~ 5 페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            store_page = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(store_page).send_keys(Keys.ENTER)
            sleep(1)
            
            html = driver.page_source
            sleep(1)
            soup = bs(html, 'html.parser')
            store_lst = soup.select('.placelist > .PlaceItem')  # 2 ~ 5 page에 검색된 가게 목록
            
            crawl_store(region, store_lst)
            
    except ElementNotInteractableException:  # 클릭할 수 없는 경우
        print('not found')
    finally:
        search_box.clear()
        
        
def crawl_store(region, store_lst):
    region=region
    
    while_flag = False
    for i, store in enumerate(store_lst):
        # 광고에 따라 index 조정
        if i >= 3:
            i += 1
        
        store_name = store.select('.head_item > .tit_name > .link_name')[0].text  # store name
        detail_page = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        # detail_page = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 최근 열린 탭으로 전환
        sleep(1)
        
        print("####", store_name)
        
        # 첫 페이지
        crawl_review(region, store_name)
        
        # 2-5 페이지
        idx = 3
        try:
            n_page = len(driver.find_elements_by_class_name('link_page'))  # 페이지 수 찾기
            for i in range(n_page-1):  # 마지막 link_page는 '다음' 버튼
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                crawl_review(region, store_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER)  # 5 페이지가 넘는 경우 '다음' 버튼 누르기
            sleep(1)
            crawl_review(region, store_name)  # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")
            
        # 그 이후 페이지
        while True:
            idx = 4
            try:
                n_page = len(driver.find_elements_by_class_name('link_page'))
                for i in range(n_page-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    crawl_review(region, store_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER)  # 10 페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                crawl_review(region, store_name)  # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0]) # 검색 탭으로 전환

def crawl_review(region, store_name):
    try:
        global driver, region_list, rating_list, category_list, review_list
        store_name_csv = store_name
        
        ret = True
        
        html = driver.page_source
        sleep(1)
        soup = bs(html, 'html.parser')
        driver.implicitly_wait(4)
        # region = soup.select('.placeinfo_default > .location_detail > span')[0].text.split()[1]
        region=region
        category = soup.select('.location_evaluation > .txt_location')[0].text.split()[1]
    
        # 첫 페이지 리뷰 목록 찾기
        review_lst = soup.select('.list_evaluation > li')
        
        # 리뷰가 있는 경우
        if len(review_lst) != 0:
            for i, reviews in enumerate(review_lst):
                rating = reviews.select('.grade_star > em')  # 별점
                review = reviews.select('.txt_comment > span') # 리뷰
                
                # region = reviews.select('.placeinfo_default > .txt_address')
                # store_region = store.select('.info_item > .addr > p')[0].text.split()[1]  # store address
                if len(review) != 0:
                    if len(rating) != 0:
                        rating = rating[0].text.replace('점', '')
                        review = review[0].text
                        
                    else:
                        rating = str('0')
                        review = review[0].text
                        
                        
                    region_list.append(region)
                    rating_list.append(rating)
                    category_list.append(category)
                    review_list.append(review)
                    
        else:
            print('no review in extract')
            ret = False
           
        return ret
    except IndexError as e:
        print(e)
        data = {'Region':region_list, 'Rating':rating_list, 'Category':category_list, 'Review':review_list}
        df = pd.DataFrame(data)
        df.to_csv('kakaomap_'+ region + '_' + store_name_csv + str(len(region_list)) +'.csv')
        pass
    except StaleElementReferenceException:
        pass
    except UnexpectedAlertPresentException:
        pass
           

# data = {'Region':region_list, 'Rating':rating_list, 'Category':category_list, 'Review':review_list}
# df = pd.DataFrame(data)
# # df.to_parquet('kakaomap_crawl.parquet', engine='pyarrow')
# df.to_csv('kakaomap_'+ region + '_' + store_name_csv + str(len(region_list)) +'.csv')



if __name__ =="__main__":
    main()

In [5]:
print(len(region_list))
print(len(rating_list))
print(len(category_list))
print(len(review_list))

7014
7014
7014
7014


In [3]:
data = {'Region':region_list, 'Rating':rating_list, 'Category':category_list, 'Review':review_list}
df = pd.DataFrame(data)
print(df.tail(10))

     Region Rating Category                                             Review
8374     포항      4        회                       투박한 느낌의 물회. 오래 기다려야 하는 게 단점.
8375     포항      3        회  맵지도 달지도 새콤하지않은 담백한 맛의 어촌스타일의 가자미물회~ 바싹 구워주는 가자...
8376     포항      4        회                             자연산물회 고기못잡으면 가게장사안하는 집
8377     포항      5        회                우리 가족들만 아는 맛집이었는데 수요미식회에 나오다니 젠장...
8378     포항      5        회                     tvN 수요미식회 "포항"편에 가자미물회가 소개된 식당
8379     포항      5        회  오리지날 포항식 물회입니다. 국물 없이 고추장 양념에 비벼먹는 까재미(가자미)회 입...
8380     포항      1        회  보통 생각하는 물회는 아닙니다. 더운날 시원한 물회를 먹으려고 했다가 폭망했습니다....
8381     포항      4        회          맛은 잇엇지만 시원한 국물맛을 즐기는 사람은 별로일거에요. 그래도 맛은 굿
8382     포항      3        회                                                   
8383     포항      3        회                   가자미회 & 구이 깔끔함 휴일은 기다리는 사람 많아서 no


In [4]:
df.to_csv('kakaomap_군산_첫번째_20944.csv')